In [1]:
import os
import time
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau , StepLR
from torch.utils.data import DataLoader
from torchvision.models import vit_b_16
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import os
import yaml
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, iirnotch, filtfilt
from scipy.ndimage import gaussian_filter
from skimage.transform import resize
import numpy as np
from scipy.signal import butter, lfilter, iirnotch, filtfilt
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset
from skimage.transform import resize
import torch
import random
#from utils.data_utils import createTrainTestSplit,create_k_fold_splits,analyze_checkpoints
#from utils.data_utils import plot_spectrograms
#from utils.training_utils import parallel_grid_search 
#from data.dataset import HMS_EEG_Dataset
#from data.dataset import HMS_Spectrogram_Dataset
#from data.dataset import CombinedDataset
#from models.models import  EEGNet, DeepConvNet
#from training.training import train_and_validate_eeg_manual_lr_grid_search, train_and_validate_eeg, train_and_validate_from_checkpoint
#from utils.config_loader import load_config
#from utils.logger_utils import setup_logger
from contextlib import redirect_stdout
from itertools import product
import io
import sys
from io import StringIO
from itertools import product
import numpy as np
import torch.multiprocessing as mp
import torch.distributed as dist
import torch
import argparse

In [2]:
def is_entirely_nan(eeg_id):
    eeg_data = load_train_eeg_frame(eeg_id)
    return np.isnan(eeg_data.values).all()

In [3]:
def load_train_spectr_frame(id):
    # Ensure the ID is an integer to prevent file path errors
    id = int(id)
    # Construct the file path using the integer ID
    file_path = os.path.join(TRAIN_SPECTR, f'{id}.parquet')
    # Load the spectrogram data from the specified Parquet file
    data = pd.read_parquet(file_path, engine='pyarrow')
    # Optional: Verify that the columns match expected Spectrogram columns
    if not CFG['SKIP_ASSERT']:
        assert list(data.columns) == CFG['SPECTR_COLUMNS'], 'Spectrogram columns order is not the same!'
    return data

In [4]:
# Load configuration from the YAML file
config_file = "/eng/home/koushani/Documents/Multimodal_XAI/Brain-Pattern-Identification-using-Multimodal-Classification/root/config/config.yml"
with open(config_file, 'r') as file:
    CFG = yaml.safe_load(file)

In [5]:
metadata = pd.read_csv('/data2/users/koushani/HMS_data/train.csv')
print(f"Loaded metadata from {CFG['root_dir']}")

Loaded metadata from /data2/users/koushani/HMS_data


In [6]:
# Function to check if the EEG data is entirely NaN
def is_entirely_nan(eeg_id):
    eeg_data = load_train_eeg_frame(eeg_id)
    return np.isnan(eeg_data.values).all()

def mirror_eeg(data):
    # Extract the relevant lists from the config
    LL = CFG['RL']  # The 'RL' key is actually referencing the 'LL' list
    LP = CFG['LP']
    RL = CFG['RP']  # The 'RP' key is referencing the 'LP' list
    RP = CFG['RL']  # The 'RL' key is also referencing the 'LL' list

    # Assuming feature_to_index is a dictionary that maps feature names to indices
    indx1 = [feature_to_index[x] for x in LL + LP if x in feature_to_index]
    indx2 = [feature_to_index[x] for x in RL + RP if x in feature_to_index]

    # Swap the data using the indices
    data[indx1, :], data[indx2, :] = data[indx2, :], data[indx1, :]
    
    return data


def load_train_eeg_frame(id):
    # Ensure the ID is an integer to avoid issues with file name construction
    id = int(id)
    # Construct the file path using the integer ID
    file_path = os.path.join(TRAIN_EEGS, f'{id}.parquet')
    # Load the EEG data from the specified Parquet file
    data = pd.read_parquet(file_path, engine='pyarrow')
    # Optional: Verify that the columns match expected EEG columns
    if not CFG['SKIP_ASSERT']:
        assert list(data.columns) == CFG['EEG_COLUMNS'], 'EEG columns order is not the same!'
    return data


def load_train_spectr_frame(id):
    # Ensure the ID is an integer to prevent file path errors
    id = int(id)
    # Construct the file path using the integer ID
    file_path = os.path.join(TRAIN_SPECTR, f'{id}.parquet')
    # Load the spectrogram data from the specified Parquet file
    data = pd.read_parquet(file_path, engine='pyarrow')
    # Optional: Verify that the columns match expected Spectrogram columns
    if not CFG['SKIP_ASSERT']:
        assert list(data.columns) == CFG['SPECTR_COLUMNS'], 'Spectrogram columns order is not the same!'
    return data


# Plot raw and processed spectrograms
def plot_spectrograms(raw, processed, labels, num_labels):
    """Plot raw and processed spectrograms."""
    x_ticks = np.linspace(0, processed.shape[1] - 1, num_labels).astype(int)
    x_labels = [labels[i] for i in x_ticks]

    plt.figure(figsize=(40, 16))

    plt.subplot(1, 2, 1)
    plt.title("Raw Signal")
    plt.imshow(raw, aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=90)
    plt.gca().xaxis.set_tick_params(labelsize=10)
    plt.gcf().subplots_adjust(bottom=0.3)

    plt.subplot(1, 2, 2)
    plt.title("Processed Signal")
    if processed.ndim == 3 and processed.shape[2] > 1:
        plt.imshow(processed[:, :, 0], aspect='auto', cmap='viridis')
    else:
        plt.imshow(processed.squeeze(), aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.xticks(ticks=x_ticks, labels=x_labels, rotation=90)
    plt.gca().xaxis.set_tick_params(labelsize=10)
    plt.gcf().subplots_adjust(bottom=0.3)

    plt.tight_layout()
    plt.show()


def baseline_correction(sig):
    sig -= np.mean(sig, axis=0)
    return sig

def normalize_signal(sig):
    """Normalize the signal by scaling it to the range [0, 1], handling NaN values."""
    sig = np.nan_to_num(sig, nan=np.nanmean(sig))
    return (sig - np.min(sig)) / (np.max(sig) - np.min(sig) + 1e-6)

def apply_notch_filter(sig, freq=60, fs=200, quality=30):
    b, a = iirnotch(freq, quality, fs)
    sig = filtfilt(b, a, sig, axis=0)
    return sig

def smooth_spectrogram(sig, sigma=1.0):
    sig = gaussian_filter(sig, sigma=sigma)
    return sig

def resample_spectrogram(sig, target_shape):
    sig = resize(sig, target_shape, mode='reflect', anti_aliasing=True)
    return sig

def handle_nan(data):
    data = data[~np.isnan(data).all(axis=1)]
    if data.size == 0:
        data = np.zeros_like(data)
    else:
        where_nan = np.isnan(data)
        mean_values = np.nanmean(data, axis=1, keepdims=True)
        mean_values[np.isnan(mean_values)] = 0
        data[where_nan] = np.take(mean_values, np.where(where_nan)[0])
    return data

def pad_or_truncate(data, length):
    if isinstance(length, int):
        if data.shape[1] < length:
            padding = np.zeros((data.shape[0], length - data.shape[1]))
            data = np.hstack((data, padding))
        else:
            data = data[:, :length]
    elif isinstance(length, tuple):
        target_rows, target_cols = length
        if data.shape[0] < target_rows:
            row_padding = np.zeros((target_rows - data.shape[0], data.shape[1]))
            data = np.vstack((data, row_padding))
        else:
            data = data[:target_rows, :]
        if data.shape[1] < target_cols:
            col_padding = np.zeros((data.shape[0], target_cols - data.shape[1]))
            data = np.hstack((data, col_padding))
        else:
            data = data[:, :target_cols]
    return data

def calculate_differential_signals(data):
    """
    Calculate differential signals based on pairs of features and concatenate them to the original data.
    
    Parameters:
    - data: numpy.ndarray, shape (n_channels, n_samples)
        The EEG data to process.
    - map_features: list of tuples
        Each tuple contains a pair of feature names (feat_a, feat_b) where the differential signal is calculated as data[feat_a] - data[feat_b].
    - feature_to_index: dict
        A dictionary mapping feature names to their respective indices in `data`.
    
    Returns:
    - combined_data: numpy.ndarray
        The original data concatenated with the calculated differential signals.
    """
    num_pairs = len(CFG['map_features'])
    differential_data = np.zeros((num_pairs, data.shape[1]))
    
    for i, (feat_a, feat_b) in enumerate(CFG['map_features']):
        if feat_a in feature_to_index and feat_b in feature_to_index:
            differential_data[i, :] = data[feature_to_index[feat_a], :] - data[feature_to_index[feat_b], :]
        else:
            print(f"Skipping: Feature {feat_a} or {feat_b} not found in feature_to_index")
            differential_data[i, :] = np.zeros(data.shape[1])

    # Debugging: Print the shapes before concatenation
    #print(f"Shape of data: {data.shape}")
    #print(f"Shape of differential_data: {differential_data.shape}")

    # Ensure that the shapes match before concatenation 
    combined_data = np.vstack((data, differential_data))
    return combined_data
    

def butter_bandpass(CFG):
    nyquist = 0.5 * CFG['sampling_rate']
    low = CFG['bandpass_filter']['low'] / nyquist
    high = CFG['bandpass_filter']['high'] / nyquist
    return butter(CFG['bandpass_filter']['order'], [low, high], btype='band')

def butter_bandpass_filter(data, CFG):
    b, a = butter_bandpass(CFG)
    return lfilter(b, a, data, axis=1)



def normalize(data):
        mean = np.mean(data, axis=1, keepdims=True)
        std = np.std(data, axis=1, keepdims=True)
        return (data - mean) / (std + 1e-6)  # Adding epsilon to avoid division by zero


def denoise_filter(x, CFG):
    y = butter_bandpass_filter(x, CFG)
    y = (y + np.roll(y, -1) + np.roll(y, -2) + np.roll(y, -3)) / 4
    y = y[:, 0:-1:4]
    return y



# data = select_and_map_channels(data, self.cfg['eeg_features'], self.feature_to_index, self.differential_channels_start_index)
def select_and_map_channels(data, channels, differential_channels_start_index):
    selected_indices = [feature_to_index[ch] for ch in channels if ch in feature_to_index]
    differential_indices = list(range(differential_channels_start_index, differential_channels_start_index + len(CFG['map_features'])))
    selected_data = data[selected_indices + differential_indices, :]
    return selected_data

def labels_to_probabilities(labels, num_classes):
    labels = torch.eye(num_classes)[labels]
    return labels


def load_checkpoint(checkpoint_dir, checkpoint_filename, model, optimizer, new_checkpoint=False):
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_filename)
    
    if not new_checkpoint and os.path.isfile(checkpoint_path):
        print(f"Loading checkpoint '{checkpoint_path}'")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        start_epoch = checkpoint['epoch']
        train_losses = checkpoint['train_losses']
        valid_losses = checkpoint['valid_losses']
        train_accuracies = checkpoint['train_accuracies']
        valid_accuracies = checkpoint['valid_accuracies']
        lr_scheduler = checkpoint['lr_scheduler']
        print(f"Loaded checkpoint from epoch {start_epoch}")
    else:
        if new_checkpoint:
            print(f"Creating a new checkpoint at '{checkpoint_path}'")
        else:
            print(f"No checkpoint found at '{checkpoint_path}', starting fresh.")
        start_epoch = 0
        train_losses = []
        valid_losses = []
        train_accuracies = []
        valid_accuracies = []
        lr_scheduler = []

    return start_epoch, train_losses, valid_losses, train_accuracies, valid_accuracies, lr_scheduler

def save_checkpoint(state, checkpoint_dir, checkpoint_filename):
    checkpoint_path = os.path.join(checkpoint_dir, checkpoint_filename)
    torch.save(state, checkpoint_path)
    print(f"Checkpoint saved at '{checkpoint_path}'")


def detect_and_save_checkpoint(state, checkpoint_dir, optimizer, regularization_lambda):
    # Detect changes in optimizer and regularization parameter
    optimizer_changed = CFG['last_optimizer'] is None or type(optimizer) != CFG['last_optimizer']
    regularization_changed = CFG.last_regularization_lambda is None or regularization_lambda != CFG['last_regularization_lambda']

    # Initialize the checkpoint filename
    checkpoint_filename = "checkpoint.pth.tar"

    # Modify the checkpoint filename based on the changes detected
    if optimizer_changed and regularization_changed:
        checkpoint_filename = "checkpoint_optimizer_and_regularization.pth.tar"
    elif optimizer_changed:
        checkpoint_filename = "checkpoint_optimizer.pth.tar"
    elif regularization_changed:
        checkpoint_filename = "checkpoint_regularization.pth.tar"

    if optimizer_changed or regularization_changed:
        print(f"Changes detected in {'optimizer' if optimizer_changed else ''} {'and' if optimizer_changed and regularization_changed else ''} {'regularization parameter' if regularization_changed else ''}. Creating a new checkpoint.")
        CFG['last_optimizer'] = type(optimizer)
        CFG['last_regularization_lambda'] = regularization_lambda
        save_checkpoint(state, checkpoint_dir, checkpoint_filename)
        
        
def create_k_fold_splits(metadata, n_splits=5):
    # Drop unnecessary columns
    metadata.drop(columns=[
            'eeg_sub_id',
            'spectrogram_sub_id',
            'patient_id',
            'label_id'
        ], inplace=True)

    # Ensure correct data types
    metadata['eeg_id'] = metadata['eeg_id'].astype(int)
    metadata['spectrogram_id'] = metadata['spectrogram_id'].astype(int)
    metadata['eeg_label_offset_seconds'] = metadata['eeg_label_offset_seconds'].astype(int)
    metadata['spectrogram_label_offset_seconds'] = metadata['spectrogram_label_offset_seconds'].astype(int)

    # Debugging: Sample the data if in DEBUG mode to reduce size for faster processing
    if CFG['debug']:
        metadata = metadata.sample(min(CFG['debug_input_size'], len(metadata)))

    # Extract features and labels for stratification
    X = metadata['eeg_id']
    y = metadata['expert_consensus']  # Correct column name for class labels

    # Create stratified K-Folds
    skf = StratifiedKFold(n_splits=n_splits)
    fold_indices = []

    for train_index, valid_index in skf.split(X, y):
        train_ids = X.iloc[train_index].tolist()
        valid_ids = X.iloc[valid_index].tolist()
        fold_indices.append((train_ids, valid_ids))

    return fold_indices

def createTrainTestSplit(metadata, fold_indices, fold_idx):
    train_ids, valid_ids = fold_indices[fold_idx]

    train_metadata = metadata[metadata['eeg_id'].isin(train_ids)]
    valid_metadata = metadata[metadata['eeg_id'].isin(valid_ids)]

    return train_metadata, valid_metadata



def seed_everything():
    np.random.seed(CFG['seed'])
    torch.manual_seed(CFG['seed'])
    random.seed(CFG['seed'])
    if torch.cuda.is_available():
        torch.cuda.manual_seed(CFG['seed'])
        torch.cuda.manual_seed_all(CFG['seed'])
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def get_device():
    if torch.cuda.is_available():
    # Get the current default CUDA device
        device = torch.device("cuda")
    # Get the name of the device
        device_name = torch.cuda.get_device_name(device)
        print(f"CUDA is available. Using device: {device} ({device_name})")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU.")
    return device


def stop_checkpointing():
    CFG['checkpointing_enabled'] = False
    print("Checkpointing disabled.")

def start_checkpointing():
    CFG['checkpointing_enabled'] = True
    print("Checkpointing enabled.")


def calculate_metrics(model, dataloader, device):
    """
    Calculate precision, recall, and F1 scores for a given model and dataloader.
    
    Args:
    - model (torch.nn.Module): Trained model.
    - dataloader (torch.utils.data.DataLoader): DataLoader for the validation/test data.
    - device (torch.device): Device to perform calculations on (CPU/GPU).
    
    Returns:
    - precision (float): Precision score.
    - recall (float): Recall score.
    - f1 (float): F1 score.
    """
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, preds = torch.max(outputs, 1)
            _, labels_max = torch.max(labels, 1)
            
            all_labels.extend(labels_max.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return precision, recall, f1

def plot_metrics(train_metrics, valid_metrics, metric_name, save_dir=CFG['save_dir']):
    """
    Plot and save the training and validation metrics over epochs.
    
    Args:
    - train_metrics (list): List of training metrics (precision, recall, or F1).
    - valid_metrics (list): List of validation metrics (precision, recall, or F1).
    - metric_name (str): Name of the metric being plotted.
    - save_dir (str): Directory to save the generated plot.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(train_metrics, label=f'Train {metric_name}')
    plt.plot(valid_metrics, label=f'Validation {metric_name}')
    plt.title(f'{metric_name} Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel(metric_name)
    plt.legend()
    plt.grid(True)
    
    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Save the plot with the specified metric name
    plot_path = os.path.join(save_dir, f'EEG_MODEL_{metric_name}.png')
    plt.savefig(plot_path)
    plt.close()

    print(f"{metric_name} plot saved at {plot_path}")


def plot_learning_rate_and_regularization(lr_scheduler, regularization_losses, save_dir=CFG['save_dir']):
    """
    Plot and save the learning rate schedule and regularization loss over epochs.

    Args:
    - lr_scheduler (list): List of learning rates over epochs.
    - regularization_losses (list): List of regularization losses over epochs.
    - save_dir (str): Directory to save the generated plot.
    """
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.plot(lr_scheduler)
    plt.title('Learning Rate Schedule')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')

    plt.subplot(1, 2, 2)
    plt.plot(regularization_losses)
    plt.title('Regularization Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Regularization Loss')

    plt.tight_layout()

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save the plot with the specified name
    plot_path = os.path.join(save_dir, 'EEG_MODEL_LearningRate_and_Regularization.png')
    plt.savefig(plot_path)
    plt.close()

    print(f"Learning Rate and Regularization Loss plot saved at {plot_path}")

def plot_accuracies(train_accuracies, valid_accuracies, save_dir=CFG['save_dir']):
    """
    Plot and save the training and validation accuracies over epochs.

    Args:
    - train_accuracies (list): List of training accuracies over epochs.
    - valid_accuracies (list): List of validation accuracies over epochs.
    - save_dir (str): Directory to save the generated plot.
    """
    epochs = range(1, len(train_accuracies) + 1)

    plt.figure(figsize=(10, 5))
    
    plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
    plt.plot(epochs, valid_accuracies, 'r-', label='Validation Accuracy')
    
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.tight_layout()

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save the plot with the specified name
    plot_path = os.path.join(save_dir, 'EEG_MODEL_Accuracy.png')
    plt.savefig(plot_path)
    plt.close()

    print(f"Training and Validation Accuracy plot saved at {plot_path}")
    
    
    
def plot_confusion_matrix(y_true, y_pred, classes, save_dir=CFG['save_dir'], normalize=False):
    """
    Plot and save the confusion matrix.

    Args:
    - y_true (list): True labels.
    - y_pred (list): Predicted labels.
    - classes (list): List of class names.
    - save_dir (str): Directory to save the generated plot.
    - normalize (bool): If True, normalize the confusion matrix.
    """
    cm = confusion_matrix(y_true, y_pred)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d", cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.tight_layout()

    # Ensure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Save the plot with the specified name
    plot_path = os.path.join(save_dir, 'Confusion_Matrix.png')
    plt.savefig(plot_path)
    plt.close()

    print(f"Confusion matrix plot saved at {plot_path}")

def create_confusion_matrix(model, dataloader, classes, device=CFG['device'], checkpoint_dir=CFG['checkpoint_dir'], checkpoint_filename='eeg_checkpoint.pth.tar', save_dir=CFG['save_dir']):
    """
    Load checkpoint, make predictions on the validation set, and plot the confusion matrix.

    Args:
    - model (torch.nn.Module): The model architecture.
    - dataloader (torch.utils.data.DataLoader): DataLoader for validation data.
    - classes (list): List of class names.
    - device (str): Device to perform computation on ('cuda' or 'cpu').
    - checkpoint_dir (str): Directory containing the checkpoint.
    - checkpoint_filename (str): Filename of the checkpoint.
    - save_dir (str): Directory to save the generated plot.
    """
    optimizer = torch.optim.Adam(model.parameters())  # Dummy optimizer to load state dict
    start_epoch, train_losses, valid_losses, train_accuracies, valid_accuracies, lr_scheduler, regularization_losses = load_checkpoint(checkpoint_dir, checkpoint_filename, model, optimizer)

    model.to(device)
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True).squeeze()  # Get the index of the max log-probability
            y_true.extend(target.cpu().numpy())
            y_pred.extend(pred.cpu().numpy())

    plot_confusion_matrix(y_true, y_pred, classes, save_dir=save_dir)
    
def load_checkpoint_for_analysis(checkpoint_path):
    if os.path.isfile(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        valid_accuracies = checkpoint['valid_accuracies']  # Extract validation accuracies
        return valid_accuracies
    else:
        print(f"No checkpoint found at {checkpoint_path}")
        return None

def analyze_checkpoints(checkpoint_dir):

    best_valid_acc = 0
    best_gamma = None
    best_step_size = None

    # Loop through all model files in the directory
    for filename in os.listdir(checkpoint_dir):
        if filename.startswith("model_"):
            # Extract gamma and step_size from filename (assumes filenames contain gamma and decay)
            parts = filename.split("_")
            gamma = float(parts[2])
            step_size = int(parts[4].split(".")[0])  # Extract the number before .pth
        
            # Load the checkpoint
            checkpoint_path = os.path.join(checkpoint_dir, filename)
            valid_accuracies = load_checkpoint_for_analysis(checkpoint_path)

        if valid_accuracies:
                # Get the maximum validation accuracy from this run
            max_valid_acc = max(valid_accuracies)
            print(f"Model: {filename} | Max Validation Accuracy: {max_valid_acc}")

                # Compare with the best validation accuracy
            if max_valid_acc > best_valid_acc:
                best_valid_acc = max_valid_acc
                best_gamma = gamma
                best_step_size = step_size

    print(f"Best model found with gamma={best_gamma}, step_size={best_step_size}, validation accuracy={best_valid_acc}")

In [7]:
feature_to_index = {x: y for x, y in zip(CFG['eeg_features'], range(len(CFG['eeg_features'])))}
ROOT_DIR = '/data2/users/koushani/HMS_data'
TRAIN_EEGS = os.path.join(ROOT_DIR , 'train_eegs')
TRAIN_SPECTR =  os.path.join(ROOT_DIR, 'train_spectrograms')
TEST_EEGS = os.path.join(ROOT_DIR, 'test_eegs')
TEST_SPECTR = os.path.join(ROOT_DIR, 'test_spectrograms')

In [8]:
metadata = pd.read_csv('/data2/users/koushani/HMS_data/train.csv')
print(f"Loaded metadata from {CFG['root_dir']}")

Loaded metadata from /data2/users/koushani/HMS_data


In [9]:
fold_indices = create_k_fold_splits(metadata, n_splits=5)
print("Created K-Fold splits.")
    

for fold_idx in range(len(fold_indices)):
    train_metadata, valid_metadata = createTrainTestSplit(metadata, fold_indices, fold_idx)
    
    print(f"Processing fold {fold_idx + 1}/{len(fold_indices)}")

Created K-Fold splits.
Processing fold 1/5
Processing fold 2/5
Processing fold 3/5
Processing fold 4/5
Processing fold 5/5


In [10]:
# HMS_Spectrogram_Dataset class
class HMS_Spectrogram_Dataset(Dataset):
    def __init__(self, train_ids, cfg, augmentations=None, plot=False):
        super(HMS_Spectrogram_Dataset, self).__init__()
        self.train_ids = train_ids
        self.cfg = cfg
        self.augmentations = augmentations
        self.plot = plot  # Flag to control plotting

    def __len__(self):
        return len(self.train_ids)

    def __getitem__(self, idx):
        row = self.train_ids.iloc[idx]
        spec_id = row['spectrogram_id']
        raw_spectrogram = load_train_spectr_frame(spec_id)
        label_name = row['expert_consensus']
        label_idx = self.cfg['name2label'][label_name]
        label = labels_to_probabilities(label_idx, self.cfg['n_classes'])
        offset = row.get("spectrogram_label_offset_seconds", None)

        if isinstance(raw_spectrogram, pd.DataFrame):
            raw_spectrogram = raw_spectrogram.to_numpy()

        if offset is not None:
            offset = offset // 2
            basic_spectrogram = raw_spectrogram[:, offset:offset + 300]
            pad_size = max(0, 300 - basic_spectrogram.shape[1])
            basic_spectrogram = np.pad(basic_spectrogram, ((0, 0), (0, pad_size)), mode='constant')
        else:
            basic_spectrogram = raw_spectrogram

        spectrogram = basic_spectrogram.T
        processed_spectrogram = pad_or_truncate(spectrogram, self.cfg['image_size'])
        processed_spectrogram = handle_nan(processed_spectrogram)
        processed_spectrogram = baseline_correction(processed_spectrogram)
        processed_spectrogram = apply_notch_filter(processed_spectrogram)
        processed_spectrogram = smooth_spectrogram(processed_spectrogram)
        processed_spectrogram = normalize_signal(processed_spectrogram)
        processed_spectrogram = resample_spectrogram(processed_spectrogram, self.cfg['image_size'])

        processed_spectrogram = np.tile(processed_spectrogram[..., None], (1, 1, 3))
        if self.plot:
            plot_spectrograms(basic_spectrogram, processed_spectrogram, self.train_ids.index.tolist(), num_labels=10)

        if self.augmentations:
            processed_spectrogram = (processed_spectrogram * 255).astype(np.uint8)
            augmented = self.augmentations(image=processed_spectrogram)
            processed_spectrogram = augmented['image']
            processed_spectrogram = processed_spectrogram.float() / 255.0
        else:
            processed_spectrogram = processed_spectrogram.astype(np.float32)
            processed_spectrogram = torch.tensor(processed_spectrogram).permute(2, 0, 1).float()

        return processed_spectrogram, label

In [11]:
spec_train_dataset = HMS_Spectrogram_Dataset(train_metadata, cfg=CFG)
print(f"Size of the train dataset: {len(spec_train_dataset)}")

spec_valid_dataset = HMS_Spectrogram_Dataset(valid_metadata, cfg=CFG)
print(f"Size of the valid dataset: {len(spec_valid_dataset)}")

Size of the train dataset: 85649
Size of the valid dataset: 21529


In [12]:
spec_train_loader = DataLoader(spec_train_dataset, batch_size=CFG['batch_size'], shuffle=True, num_workers=CFG['num_workers'], pin_memory=True)
spec_valid_loader = DataLoader(spec_valid_dataset, batch_size=CFG['batch_size'], shuffle=True, num_workers=CFG['num_workers'], pin_memory=True)

In [13]:
start_time = time.time()
# Iterate over batches and log the shape of the final DataLoader object
for batch in spec_train_loader:
    data, labels = batch
    print(f"Batch data shape: {data.shape}")  # Logs batch size and dimensions
    print(f"Batch labels shape: {labels.shape}")  # Logs label dimensions
    break
end_time = time.time()
duration = end_time - start_time
print(f"took {duration} seconds to display dataloader info.")


KeyboardInterrupt: 

In [ ]:
class SpectrogramViT(nn.Module):
    def __init__(self, image_size=(400, 300), num_classes=6):
        super(SpectrogramViT, self).__init__()
        
        # Initialize a pre-trained Vision Transformer with ImageNet weights
        self.vit = vit_b_16(pretrained=True)
        
        # Calculate the number of patches based on the new image size
        new_patch_size = (image_size[0] // 16, image_size[1] // 16)
        num_patches = new_patch_size[0] * new_patch_size[1]
        
        # Adjust the positional embeddings to match the new number of patches
        self.vit.encoder.positional_embedding = nn.Parameter(
            torch.randn(1, num_patches + 1, 768)
        )
        
        # Get the input features from the classifier head
        in_features = self.vit.heads[0].in_features
        
        # Adjust the classification head to output logits for KLDivLoss
        self.vit.heads = nn.Sequential(
            nn.Linear(in_features, num_classes),
            nn.LogSoftmax(dim=1)  # Use LogSoftmax for KLDivLoss
        )

    def forward(self, x):
        return self.vit(x)

In [ ]:
# Create the ViT model
model = SpectrogramViT(image_size=(400, 300), num_classes=6)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(model)

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


SpectrogramViT(
  (vit): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): La

In [ ]:
# Loss function adjusted for KL divergence
criterion = nn.KLDivLoss(reduction='batchmean')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training function remains the same, but ensure the labels are in probability form
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for data in train_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Ensure labels are a probability distribution if using KLDivLoss
            labels = labels / labels.sum(dim=1, keepdim=True)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels.argmax(dim=1)).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100 * correct / total

        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

        # Validation Phase
        model.eval()
        with torch.no_grad():
            val_loss, val_correct, val_total = 0.0, 0, 0
            for data in val_loader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                labels = labels / labels.sum(dim=1, keepdim=True)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels.argmax(dim=1)).sum().item()

            val_loss = val_loss / len(val_loader.dataset)
            val_acc = 100 * val_correct / val_total
            print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%')

    print('Training complete.')

In [ ]:
train_model(model, spec_train_loader, spec_valid_loader, criterion, optimizer, num_epochs=10)